In [2]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType
import requests
import pandas as pd
from dotenv import load_dotenv
import os

cmf_param = {
    'chain_id': 1,
}

context, _model_loader = create_cmf(cmf_param)

In [32]:
rain_mainnet_factory = Contract('0x389c6f04D39fF038cC99F94BcE338D2f5Eb64a67')
rain_factory_proxy = Contract('0x31EBf70312f488D0bdAc374b340f0D01dBf153B5')

rain_controller = Contract('0xE5D3d7da4b24bc9D2FDA0e206680CD8A00C0FeBD')
usdc_addr = Token("USDC")

load_dotenv()

RAIN_DEVELOPER_API_KEY = os.environ.get('RAIN_DEVELOPER_API_KEY')
RAIN_CARD_SPEND_ENDPOINT = os.environ.get('RAIN_CARD_SPEND_ENDPOINT')


start_date = BlockNumber.from_ymd(2023, 4, 21)
current_date=context.block_number
usdc_addr.functions.

In [29]:
# rain_mainnet_factory.ledger.events.NewRainCollateral
rain_factory_proxy.ledger.events.NewRainCollateral

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__name', 'evt__user'])

In [38]:
with rain_factory_proxy.ledger.events.NewRainCollateral as q:
    res = q.select(columns=[q.EVT__COLLATERALPROXY],
                    order_by=q.BLOCK_NUMBER.desc(),
                    limit=1000).to_dataframe()
    
print(res.columns)

# iterate through each row in res

# for each row, get the balance of the collateral proxy

for index, row in res.iterrows():
    # print(row['evt__collateralProxy'])
    collateral_proxy = Address(row['evt__collateralProxy'])
    balance = usdc_addr.functions.balanceOf(collateral_proxy).call()
    print(balance)
    # with collateral_proxy.balance_of(usdc_addr) as q:
    #     res = q.select(columns=[q.EVT__VALUE],
    #                 order_by=q.BLOCK_NUMBER.desc(),
    #                 limit=200).to_dataframe()
    # print(res)
    # # iterate through each row in res
    # # for each row, get the balance of the collateral proxy
    # for index, row in res.iterrows():
    #     print(row['VALUE'])
    #     # if the balance is greater than 0, then we need to call the rain card spend endpoint
    #     if row['VALUE'] > 0:
    #         print('calling rain card spend endpoint')
    #         # call the rain card spend endpoint



Index(['evt__collateralProxy'], dtype='object')
0
0
1800000000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9000000000
0
0
0
0
0
74465700000
0
0
0
0
0
0
0
0
0
0
0
0
948550000
0
0
0
0
0
0
0
0
0
0
0


In [75]:
rain_controller.ledger.events.Liquidation

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__assets', 'evt__amounts'])

In [ ]:
with rain_controller.ledger.events.Liquidation as q:
    liquidations = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__AMOUNTS
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(liquidations)

In [76]:
rain_controller.ledger.events.Payment

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__assets', 'evt__amounts'])

In [ ]:
with rain_controller.ledger.events.Payment as q:
    payments = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__AMOUNTS
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(payments)

In [79]:
rain_controller.ledger.events.Withdrawal

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__asset', 'evt__amount'])

In [ ]:
with rain_controller.ledger.events.Withdrawal as q:
    withdrawals = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__AMOUNT
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(withdrawals)

In [39]:
def get_card_spend_dataframe():
    # Headers (if required)
    headers = {
        'API-Key': RAIN_DEVELOPER_API_KEY,
        'accept': 'application/json'
    }

    # Make the GET request
    response = requests.get(RAIN_CARD_SPEND_ENDPOINT, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Process the response if successful
        data = response.json()
        print(data)
    else:
        # Handle errors (you can print or log the error response)
        print(f"Error: {response.status_code}")

    # create dataframe
    df = pd.DataFrame([data])

    # add date column
    df.insert(0, 'date', current_date)


    # drop dueBalance
    df = df.drop(columns=['dueBalance'])
    
    # add totalBalance
    df['totalBalance'] = df['pendingBalance'] + df['postedBalance']
    
    return df

In [21]:
latest_card_spend_df = get_card_spend_dataframe()

{'pendingBalance': 598753, 'postedBalance': 45042271, 'dueBalance': 127662}
